# DynaFu ICP

DynaFu's ICP works more or less like KinFu's ICP so first let's remember how KinFu's ICP works.

## KinFu ICP

### Common principles

First of all, KinFu's ICP takes two frames - new and old, or camera's frame and global frame.
Each frame has the same size as input depth map and consists of 3D points and normals for each pixel.
Camera's frame is generated from depth map from camera, global frame generated from currently observed volume from previous camera pose.

The task is to find such 3D Rt transform that aligns camera's frame to global frame.

This Rt transform is obtained one pyramid level by one, one iterative approximation of Rt by one.

Each camera's frame pixel is compared to the global pixel in which it projects to.

### Metric

The metric is calculated across all valid pixels using point-plane metrics:

$
\begin{equation*}
E = \sum_{u \in \Omega}  \left \| \left( T_{c2g} V_c^u - V_g^u  \right)^T N_g^u  \right \|_2
\end{equation*}
$

where $V_c^u$ is a camera's frame vertex at pixel u, $N_g^u$ is a normal in the global frame.

$T_{c2g}$ is an approximation of our Rt transform at current iteration.

Here $\Omega$ represents all valid pixels on which we calculate function.
The conditions are the following:
* all points and normals values are valid (not NaN or Inf)
* transformed $V_c$ is not too far from $V_g$ (tunable algorithm parameter)
* the angle between $N_g$ and transformed $N_c$ is not too small (tunable algorithm parameter, usually 10-30 degrees)

### Optimization

So how to minimize $E$ (or its square which is mathematically simplier)? Using iterations and approximations!

The procedure is a Gauss-Newton algorithm (or at least resembles it in some details).

Let's represent $E$ as a function of $x$ - transform's parameters. Let  $f$ be a shift from $V_c$ to new place: $ f(x, V_c) = T_{c2g}V_c^u - V_c^u$

We remember that the derivative is zero in error's minimum:

$
\begin{equation*}
J\left(E(x_0)\right) = \left [
\frac{\partial E(x)}{\partial x_1} \mid
\frac{\partial E(x)}{\partial x_2} \mid \cdots
\right ] =
0_{1 \times n=length(x)}
\end{equation*}
$

and let's calc this derivative itself. Long story short, we have:

$
\begin{equation*}
\frac{\partial E}{\partial x_i} = \sum_{u \in \Omega} 2 \cdot \underbrace{N_g^T}_{1 \times 3} \cdot \underbrace{\left( f(x, V_c) + V_c - V_g \right)}_{3 \times 1} \cdot \underbrace{N_g^T}_{1 \times 3} \cdot \underbrace{\left [ \frac{\partial f(x, V_c)}{\partial x_i} \right ]}_{3 \times 1} = 0 \\
\end{equation*}
$

The Jacobian of function f (i.e. the matrix of all derivatives) will be:

$
f'_{x_i}(x, V_c) = \left [ \frac{\partial f(x, V_c)}{\partial x_i} \right ] \\
J_f = \underbrace{\left[ f'_{x_1}(x, V_c) \mid f'_{x_2}(x, V_c) \mid \cdots \right]}_{n=length(x)}
$

Let $V_{\Delta} = V_c - V_g$ and perform more calculations:

$
\begin{equation*}
\sum_{u \in \Omega} 2 \cdot N_g^T \cdot (f(x, V_c) + V_{\Delta}) \cdot N_g^T \cdot f'_{x_i}(x, V_c) = 0 \\
\end{equation*}
$

$
\begin{equation*}
\sum_{u \in \Omega} \left( 2 \cdot N_g^T \cdot f(x, V_c) \cdot N_g^T \cdot f'_{x_i}(x, V_c) \right) + 
\sum_{u \in \Omega} \left( 2 \cdot N_g^T \cdot V_{\Delta} \cdot N_g^T \cdot f'_{x_i}(x, V_c) \right)= 0 \\
\end{equation*}
$

$
\begin{equation*}
\sum_{u \in \Omega} N_g^T \cdot f(x, V_c) \cdot N_g^T \cdot f'_{x_i}(x, V_c) =
\sum_{u \in \Omega} N_g^T \cdot (-V_{\Delta}) \cdot N_g^T \cdot f'_{x_i}(x, V_c)  \\
\end{equation*}
$

$
\begin{equation*}
\sum_{u \in \Omega} f(x, V_c)^T \cdot (N_g \cdot N_g^T) \cdot f'_{x_i}(x, V_c) = 
\sum_{u \in \Omega} (-V_{\Delta})^T \cdot (N_g \cdot N_g^T) \cdot f'_{x_i}(x, V_c) \\
\end{equation*}
$

$
\begin{equation*}
\sum_{u \in \Omega} ((f(x, V_c)+V_{\Delta})^T \cdot (N_g \cdot N_g^T)) \cdot f'_{x_i}(x, V_c) = 0 \\
\sum_{u \in \Omega} \underbrace{((f(x, V_c)+V_{\Delta})^T (N_g N_g^T))}_{1 \times 3}  \underbrace{J_f(x, V_c)}_{3 \times n} = \underbrace{0}_{1 \times n} \\
\end{equation*}
$

$
\begin{equation*}
\sum_{u \in \Omega} J_f^T(x, V_c) ((f(x, V_c)+V_{\Delta})^T (N_g N_g^T))^T = 0 \\
\sum_{u \in \Omega} \underbrace{J_f^T(x, V_c)}_{n \times 3} (N_g N_g^T) (f(x, V_c)+V_{\Delta}) = \underbrace{\mathbf{0}}_{n \times 1} 
\end{equation*}
$

### KinFu ICP warp function

The last equation is a common for all warp functions $f$. Let's consider the warp function of KinFu.

In KinFu $f$ is not more than Rt transform: $f(x, V_c) = R(x)\cdot V_c+t$. But KinFu it uses its approximation near zero angle where $sin(\alpha) \rightarrow \alpha $ and $cos(\alpha) \rightarrow 1$. By substituting this into rotating matrix instead of real sin and cos they simply get the following approximation:

$
R_{approx} = I + \begin{bmatrix}
0 & -\gamma & \beta \\ 
\gamma & 0 & -\alpha\\ 
-\beta & \alpha & 0
\end{bmatrix} = I + R_{shift}
$

It's mentioned that this strange matrix is skew-simmetric (or anti-simmetric) and that it can be useful to express cross through it:

$
R_{shift} \cdot v_{3 \times 3} = \begin{bmatrix} \alpha \\ \beta \\ \gamma \end{bmatrix} \times v_{3 \times 3}
$

Therefore the notation is the following: $R_{shift} = \begin{bmatrix} \alpha \\ \beta \\ \gamma \end{bmatrix}_{\bigotimes} $

and the operation can be expressed like that: $R(x) \cdot V_c = \begin{bmatrix} \alpha \\ \beta \\ \gamma \end{bmatrix} \times V_c $ 

Using cross product property that $ a \times b = - (b \times a) $ we can write:

$
R(x) \cdot V_c = [V_c]_{\bigotimes} \times \begin{bmatrix} \alpha \\ \beta \\ \gamma \end{bmatrix}
$

In fact, KinFu uses 6 params for x: 3 for translation (which are applied the obvious way) and 3 for rotation. 3 for rotation is rotation axis multiplied by an angle of rotation which refers to Rodrigues formula.

Getting $R_approx$ refers also to tangent space of rotation manifold, getting rotation from that tangent space refers to exponential map of that manifold or something about that. These are topics which I don't understand quite good yet but I recommend to explore them.

***To read: Rodrigues formula, [3D rotation group](https://en.wikipedia.org/wiki/3D_rotation_group#Exponential_map), google how to differentiate rotations.***

### Getting final linear equation

By some reason KinFu uses f the same as its derivative:
$
\begin{equation*}
f(x, V_c) = G(V_c) \cdot x = \left[[V_c]_{\bigotimes}|I_{3 \times 3}\right]\cdot x 
\end{equation*}
$

Let's substitute it to final equation and get the linear system to solve:

$
\begin{equation*}
\sum_{u \in \Omega} G^T(V_c) (N_g N_g^T) (G(V_c)\cdot x+V_{\Delta}) = \mathbf{0} \\
\sum_{u \in \Omega} G^T_{V_c} (N_g N_g^T) G_{V_c}\cdot x = \sum_{u \in \Omega} G^T_{V_c} (N_g N_g^T) (-V_{\Delta})
\end{equation*}
$

Let's introduce new notation: $ A = G^T_{V_c}N_g $

$
\begin{equation*}
\sum_{u \in \Omega} A A^T \cdot x = \sum_{u \in \Omega} A N_g^T (-V_{\Delta}) \\
\end{equation*}
$

Here we get a system of linear equations which can be solved by any known method since it's not typical for least-squares methods: it's non-singular, has enough equations, etc., in other words, it's good.

After solving that for x we use this x to update our Rt transform and proceed to next iteration.